In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding,Flatten, Dense
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import os
import numpy as np

Using TensorFlow backend.


In [2]:
max_features = 10000
max_len = 200

In [3]:
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=max_features)
words_index = imdb.get_word_index()

In [4]:
print(len(words_index))

88584


In [5]:
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [6]:
x_train.shape

(25000, 200)

In [7]:
glove_dir = '/home/a/diskdata/glove.6B'

In [8]:
embeddings_index = {}
f = open(os.path.join(glove_dir,'glove.6B.100d.txt'))

In [9]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs
f.close

<function TextIOWrapper.close()>

In [10]:
print(embeddings_index['dog'])
print(embeddings_index['the'])

[ 0.30817    0.30938    0.52803   -0.92543   -0.73671    0.63475
  0.44197    0.10262   -0.09142   -0.56607   -0.5327     0.2013
  0.7704    -0.13983    0.13727    1.1128     0.89301   -0.17869
 -0.0019722  0.57289    0.59479    0.50428   -0.28991   -1.3491
  0.42756    1.2748    -1.1613    -0.41084    0.042804   0.54866
  0.18897    0.3759     0.58035    0.66975    0.81156    0.93864
 -0.51005   -0.070079   0.82819   -0.35346    0.21086   -0.24412
 -0.16554   -0.78358   -0.48482    0.38968   -0.86356   -0.016391
  0.31984   -0.49246   -0.069363   0.018869  -0.098286   1.3126
 -0.12116   -1.2399    -0.091429   0.35294    0.64645    0.089642
  0.70294    1.1244     0.38639    0.52084    0.98787    0.79952
 -0.34625    0.14095    0.80167    0.20987   -0.86007   -0.15308
  0.074523   0.40816    0.019208   0.51587   -0.34428   -0.24525
 -0.77984    0.27425    0.22418    0.20164    0.017431  -0.014697
 -1.0235    -0.39695   -0.0056188  0.30569    0.31748    0.021404
  0.11837   -0.11319    

In [11]:
embedding_dim = 100

In [12]:
embedding_matrix = np.zeros((max_features, embedding_dim))

In [13]:
print(embedding_matrix.shape)

(10000, 100)


In [15]:
for word,i in words_index.items():
    if i < max_features:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [16]:
len(embedding_matrix[1])

100

# model

In [17]:
model = Sequential()

In [18]:
model.add(Embedding(max_features, embedding_dim,input_length=max_len))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                640032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,640,065
Trainable params: 1,640,065
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.layers[0].set_weights = ([embedding_matrix])
model.layers[0].trainable = False

In [21]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics =['acc'])

In [22]:
print(type(x_train))
print(x_train.shape)
print(len(x_train[1]))

<class 'numpy.ndarray'>
(25000, 200)
200


In [23]:
history = model.fit(x_train, y_train,
             epochs = 10,
             batch_size=100,
             validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 4s 206us/step - loss: 0.6788 - acc: 0.5688 - val_loss: 0.6551 - val_acc: 0.6138
Epoch 2/10
20000/20000 [==============================] - 4s 189us/step - loss: 0.5623 - acc: 0.7178 - val_loss: 0.6518 - val_acc: 0.6226
Epoch 3/10
20000/20000 [==============================] - 4s 194us/step - loss: 0.4530 - acc: 0.7945 - val_loss: 0.6867 - val_acc: 0.6252
Epoch 4/10
20000/20000 [==============================] - 4s 196us/step - loss: 0.3579 - acc: 0.8525 - val_loss: 0.8095 - val_acc: 0.5866
Epoch 5/10
20000/20000 [==============================] - 4s 190us/step - loss: 0.2762 - acc: 0.8944 - val_loss: 0.8289 - val_acc: 0.6082
Epoch 6/10
20000/20000 [==============================] - 4s 196us/step - loss: 0.2129 - acc: 0.9243 - val_loss: 0.8889 - val_acc: 0.6144
Epoch 7/10
20000/20000 [==============================] - 4s 194us/step - loss: 0.1661 - acc: 0.9397 - val_loss: 0.9869 - v